In [1]:
import pandas as pd
import base
import os
import re

2024-01-05 04:52:43.991008: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-01-05 04:52:44.034042: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-05 04:52:44.034084: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-05 04:52:44.035051: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-05 04:52:44.041393: I tensorflow/core/platform/cpu_feature_guar

In [2]:
SEED = 42
base.fix_seed(SEED)

desired_directory = '/home/jovyan/work/spectrum/'
os.chdir(desired_directory)

In [3]:
df_pos_ok_data2 =  pd.read_csv('Data/02_basedata/pos/pos2.csv')

In [4]:
df = pd.read_csv('Data/02_basedata/df.csv')
df_pos = df[df['Adducttype'].str.contains('\]\+') | df['Adducttype'].str.contains('\]2\+')].drop('Unnamed: 0', axis=1)
df_pos_select = df_pos.merge(df_pos_ok_data2[['Metabolitename', 'dataset', 'AlignmentID']].reset_index().rename(columns={'index':'index_ID'}),
                             on=['Metabolitename', 'dataset', 'AlignmentID'])

".ms" fileの作成

In [118]:
invalid_chars = r'[\\/:*?"<>|]'

for i in range(len(df_pos_select)):
    compound = df_pos_select['Metabolitename'][i]
    formula = df_pos_select['Formula'][i]
    parentmass = df_pos_select['AverageMz'][i]
    ionization = df_pos_select['Adducttype'][i]
    
    ms2_data = df_pos_select['MSMSspectrum'][i]
    elements = ms2_data.split()  
    id = df_pos_select.index.tolist()[i]
    
    metabolite = re.sub(invalid_chars, '_', df_pos_select['Metabolitename'][i])
    ontology = re.sub(invalid_chars, '_', df_pos_select['Ontology'][i])
    name_ont = f"{id}_{metabolite}_{ontology}"

    ms2_split_data = []
    for element in elements:
        parts = element.split(':')
        ms2_split_data.append(f"{parts[0]} {parts[1]}")

    output_file = f"Data/04_canopus/pos/ms2_spectrum_ms_pos3/{name_ont}.ms"
    with open(output_file, 'w', newline='') as file:
        file.write(f">compound {metabolite}_{ontology}\n")
        file.write(f">formula {formula}\n")
        file.write(f">parentmass {parentmass}\n")
        file.write(f">ionization {ionization}\n")
        
        file.write(">ms2\n")
        file.write('\n'.join(ms2_split_data)) 

In [119]:
import zipfile
import os
import re

folder_path = 'Data/04_canopus/pos/ms2_spectrum_ms_pos3'
zip_filename = 'Data/04_canopus/pos/ms2_spectrum_ms_pos3.zip'

with zipfile.ZipFile(zip_filename, 'w') as zipf:
    for foldername, subfolders, filenames in os.walk(folder_path):
        for filename in filenames:
            if filename.endswith('.ms'):
                file_path = os.path.join(foldername, filename)
                arcname = os.path.relpath(file_path, folder_path)
                zipf.write(file_path, arcname)

#### 読み込めた数:8400/8451

canopus resultの読み込み

In [9]:
df_canopus = pd.read_csv('Data/04_canopus/pos/canopus_compound_summary_pos2.tsv', sep="\t")
df_canopus['index_id'] = df_canopus['id'].apply(lambda x: x.split("_")[1])
df_canopus_ = df_canopus[['id','ClassyFire#subclass','ClassyFire#level 5','index_id']]
df_canopus_['index_id'] = df_canopus_['index_id'].astype(int)

/tmp/ipykernel_1375061/2445995084.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_canopus_['index_id'] = df_canopus_['index_id'].astype(int)


使うサブクラスの選択

In [5]:
select_ont = pd.read_csv('Data/04_canopus/pos/subclass_ont_select_pos.csv')

In [6]:
df_ytest = pd.read_csv('Data/04_canopus/pos/df_pos_ytest.csv').rename(columns={'Unnamed: 0':'index_ID'})
df_ytest = df_ytest[df_ytest['Ontology'].isin(select_ont.Ontology.tolist())].merge(select_ont, how='left', on='Ontology')

In [10]:
df_pos_select2 = df_pos_select.reset_index()[['index','index_ID']].rename(columns={'index':'index_id'}).astype(int)
df_model_canopus = df_pos_select2.merge(df_ytest, how='right', on='index_ID').merge(df_canopus_, how='left', on='index_id').\
drop(columns='index_id').sort_values('index_ID')

サブクラスごとの正答数を整理

In [11]:
subclasslist = ['Monoradylglycerols',
 'Ceramides',
 'Diradylglycerols',
 'Glycerophosphocholines',
 'Triradylglycerols',
 'Phosphosphingolipids',
 'Glycosphingolipids',
 'Glycerophosphoethanolamines',
 'Glycerophosphoglycerols']

df_pos_cano_class_pred = pd.DataFrame(columns=['Subclass','total', 'canopus_export','canopus_corpred', 'canopus_midpred'])

for i in subclasslist:
    df_model_canopus_i = df_model_canopus[df_model_canopus['subclass'] == i]
    export_number = len(df_model_canopus_i) - len(df_model_canopus_i[df_model_canopus_i['ClassyFire#subclass'].isna()])
    correct_predictions = len(df_model_canopus_i[df_model_canopus_i['subclass'] == df_model_canopus_i['ClassyFire#subclass']])
    mispred_number = export_number - correct_predictions
    total_samples = len(df_model_canopus_i)

    new_row = {'Subclass': i,
               'total':total_samples,
               'canopus_export': export_number,
               'canopus_corpred': correct_predictions,
               'canopus_midpred': mispred_number}
    
    df_pos_cano_class_pred = pd.concat([df_pos_cano_class_pred, pd.DataFrame([new_row])], ignore_index=True)
    
df_pos_cano_class_pred.to_csv('Data/04_canopus/pos/df_pos_canopus_subclass_pred.csv', index=False)

モデルの予測結果と並べる

In [12]:
df_pos_model_subclass_pred = pd.read_csv('Data/04_canopus/pos/df_pos_subclass_pred.csv')

In [13]:
df_pos_model_subclass_pred2 = df_pos_model_subclass_pred.merge(df_pos_cano_class_pred, how='inner', on='Subclass').drop(columns='total')
df_pos_model_subclass_pred2.to_csv('Data/04_canopus/pos/df_pos_model_canopus_subclass_pred.csv', index=False)
df_pos_model_subclass_pred2

,Subclass,Correct_Predictions,Incorrect_Predictions,Total,canopus_export,canopus_corpred,canopus_midpred
0,Monoradylglycerols,32,0,32,3,0,3
1,Ceramides,109,2,111,5,4,1
2,Diradylglycerols,207,5,212,15,12,3
3,Glycerophosphocholines,249,9,258,0,0,0
4,Triradylglycerols,410,7,417,23,0,23
5,Phosphosphingolipids,82,2,84,21,16,5
6,Glycosphingolipids,101,3,104,8,8,0
7,Glycerophosphoethanolamines,110,5,115,1,1,0
8,Glycerophosphoglycerols,12,0,12,1,1,0
